In [1]:
import os
import sqlite3
import csv
from pprint import pprint

os.chdir('C:\sqlite_windows')
sqlite_file = "project.db"
conn = sqlite3.connect(sqlite_file)
conn.text_factory = str
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes')
conn.commit()

cur.execute('''
    Create Table nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')

conn.commit()


with open('nodes.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in table]
    cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_table)

conn.commit()






In [2]:
# 2. Getting number of nodes in the table:

cur.execute("SELECT COUNT(*) FROM nodes;")
print(cur.fetchall())

[(2327433,)]


In [3]:
# 3. Forming table for ways based on ways.csv file

cur.execute('DROP TABLE IF EXISTS ways')
conn.commit()

cur.execute('''
    Create Table ways(id INTEGER, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT)
''')

conn.commit()


with open('ways.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in table]
    cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_table)

conn.commit()

In [4]:
# 4. Getting number of ways in the table:

cur.execute("SELECT COUNT(*) FROM ways;")
print(cur.fetchall())

[(238981,)]


In [5]:
# 5. Forming table for node_tags based on node_tags.csv file

cur.execute('DROP TABLE IF EXISTS nodes_tags')
conn.commit()

cur.execute('''
    Create Table nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')

conn.commit()

with open('nodes_tags.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['key'], i['value'], i['type']) for i in table]
    cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?,?,?,?);", to_table)

conn.commit()

In [6]:
# 6. Forming table for way_tags based on way_tags.csv file

cur.execute('DROP TABLE IF EXISTS ways_tags')
conn.commit()

cur.execute('''
    Create Table ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT)
''')

conn.commit()

with open('ways_tags.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['key'], i['value'], i['type']) for i in table]
    cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?,?,?,?);", to_table)

conn.commit()

In [7]:
# 7. Forming table for way_nodes based on way_nodes.csv file

cur.execute('DROP TABLE IF EXISTS ways_nodes')
conn.commit()

cur.execute('''
    Create Table ways_nodes(id INTEGER, node_id INTEGER, position INTEGER)
''')

conn.commit()

with open('ways_nodes.csv','r') as file:
    table = csv.DictReader(file)
    to_table = [(i['id'], i['node_id'], i['position']) for i in table]
    cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?,?,?);", to_table)

conn.commit()

In [8]:
cur.execute("SELECT COUNT(DISTINCT(e.uid)) \
             FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;")
print(cur.fetchall())

cur.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
             GROUP BY e.user \
             ORDER BY num DESC LIMIT 10;")

print(cur.fetchall())

[(949,)]
[('jumbanho', 1557785), ('JMDeMai', 202694), ('bdiscoe', 130371), ('woodpeck_fixbot', 113992), ('bigal945', 103428), ('yotann', 66738), ('runbananas', 41462), ('sandhill', 32488), ('MikeInRaleigh', 30730), ('Clay Hobbs', 21928)]


In [9]:
#Top 5 Cities

import pprint 

cur.execute("SELECT tags.value, COUNT(*) as count \
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags)tags \
             WHERE tags.key LIKE '%city'\
             GROUP BY tags.value \
             ORDER BY count DESC \
             LIMIT 5;")

            
pprint.pprint(cur.fetchall())

            

[('Raleigh', 5834),
 ('Cary', 2815),
 ('Morrisville', 1653),
 ('Durham', 1378),
 ('Chapel Hill', 516)]


In [10]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as num FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%religion'\
             GROUP BY tags.value \
             ORDER BY num DESC;")

pprint.pprint(cur.fetchall())

[('christian', 492),
 ('muslim', 6),
 ('jewish', 5),
 ('unitarian_universalist', 3),
 ('bahai', 1),
 ('eckankar', 1),
 ('hindu', 1),
 ('sai_baba', 1)]


In [12]:
import pprint
cur.execute ("SELECT tags.value, COUNT(*) as num FROM (SELECT * FROM nodes_tags UNION ALL \
             SELECT * FROM ways_tags) tags \
             WHERE tags.key LIKE '%amenity'\
             GROUP BY tags.value \
             ORDER BY num DESC limit 10;")

pprint.pprint(cur.fetchall())

[('parking', 2560),
 ('restaurant', 604),
 ('bicycle_parking', 577),
 ('place_of_worship', 539),
 ('fast_food', 295),
 ('school', 229),
 ('fuel', 218),
 ('bench', 134),
 ('waste_basket', 120),
 ('bank', 119)]


In [13]:
import pprint

cur.execute ("SELECT value, COUNT (*) \\

     FROM(SELECT * from nodes_tags as N UNION ALL \
          SELECT * from ways_tags as M) as O \
          WHERE (value = 'restaurant' or value = 'hotel' or \
                 value = 'pub' or value = 'bar' or \
                 value = 'cafe' or value ='bakery') \
        GROUP BY value;")
             
pprint.pprint(cur.fetchall()) 

[('bakery', 20),
 ('bar', 66),
 ('cafe', 112),
 ('hotel', 136),
 ('pub', 48),
 ('restaurant', 605)]


In [14]:
import pprint

cur.execute("SELECT tags.value, COUNT(*) as count \
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags)tags \
             WHERE tags.key LIKE '%cuisine'\
             GROUP BY tags.value \
             ORDER BY count DESC \
             LIMIT 5;")
pprint.pprint(cur.fetchall()) 

[('burger', 89),
 ('sandwich', 63),
 ('mexican', 59),
 ('pizza', 57),
 ('american', 53)]


In [14]:
import pprint
cur.execute ("SELECT value  FROM ways_tags \
             WHERE key = 'phone' \
             LIMIT 10;")

pprint.pprint(cur.fetchall())

[('+1-919-572-8808',),
 ('+1 919 286 4211',),
 ('9192861179',),
 ('+1 919 560 3912',),
 ('+1 919 560 3925',),
 ('+1 919 668 4000',),
 ('+1 919 286 4421',),
 ('+1 919 681 3937',),
 ('+1 919 286 1593',),
 ('+1 919 7972634',)]


In [29]:
import os

print "OSM File Size is:", os.path.getsize('raleigh_north-carolina.osm')*1e-6, "MB" 
print "project.db file is:", os.path.getsize('project.db')*1e-6, "MB"
print "nodes.csv file is:", os.path.getsize('nodes.csv')*1e-6, "MB"
print "nodes_tags.csv file is:", os.path.getsize('nodes_tags.csv')*1e-6, "MB"
print "ways.csv file is:", os.path.getsize('ways.csv')*1e-6, "MB"
print "ways_nodes.csv file is:", os.path.getsize('ways_nodes.csv')*1e-6, "MB"
print "ways_tags.csv file is:", os.path.getsize('ways_tags.csv')*1e-6, "MB"

OSM File Size is: 483.61722 MB
project.db file is: 275.780608 MB
nodes.csv file is: 190.804602 MB
nodes_tags.csv file is: 2.195973 MB
ways.csv file is: 14.009066 MB
ways_nodes.csv file is: 63.019839 MB
ways_tags.csv file is: 30.812796 MB
